In [1]:
from math import pi

import numpy as np
import pandas as pd

from bokeh.plotting import figure, output_notebook, show

output_notebook()

Loading BokehJS ...

# Methods

In [2]:
def get_light_frequency(note, octaves=41):
    light_frequency = note * pow(2, octaves) * 1e-12
    # print(f"name: {note['name']}\n\t light_frequency: {round(light_frequency, 2)}")
    return light_frequency

In [3]:
# Thanks: https://stackoverflow.com/questions/44959955/matplotlib-color-under-curve-based-on-spectral-color
def frequency_to_wavelength(frequency):
    """
    converts a frequency (in THz) to wavelength (in nm)
    input:
        frequency (THz)
    output:
        wavelength (nm)
    """
    c = 299792458 # m/s
    wavelength = c / (frequency * 1e12) * 1e9
    return wavelength

def wavelength_to_rgb(wavelength, gamma=0.2):
    ''' taken from http://www.noah.org/wiki/Wavelength_to_RGB_in_Python
    This converts a given wavelength of light to an 
    approximate RGB color value. The wavelength must be given
    in nanometers in the range from 380 nm through 750 nm
    (789 THz through 400 THz).

    Based on code by Dan Bruton
    http://www.physics.sfasu.edu/astro/color/spectra.html
    Additionally alpha value set to 0.5 outside range
    '''
    wavelength = float(wavelength)
    if wavelength >= 380 and wavelength <= 750:
        A = 1.
    else:
        A=0.5
    if wavelength < 380:
        wavelength = 380.
    if wavelength >750:
        wavelength = 750.
    if wavelength >= 380 and wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G = 0.0
        B = (1.0 * attenuation) ** gamma
    elif wavelength >= 440 and wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0
    elif wavelength >= 490 and wavelength <= 510:
        R = 0.0
        G = 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma
    elif wavelength >= 510 and wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G = 1.0
        B = 0.0
    elif wavelength >= 580 and wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0
    elif wavelength >= 645 and wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G = 0.0
        B = 0.0
    else:
        R = 0.0
        G = 0.0
        B = 0.0
    scale = 255
    R *= scale
    G *= scale
    B *= scale
    return (R,G,B,A)

In [4]:
def get_color(frequency):
    """
    return RGBA for given light frequency (in THz)
    """
    return wavelength_to_rgb(frequency_to_wavelength(frequency), gamma=.5)



In [5]:
def make_fig(df):
    df = df.copy()
    df['x'] = range(len(df))
    p = figure(height=300)
    p.rect(x='x', y=1, width=1, height=1, color='color', source=df)
    p.text(
        x='x', y=.5, text='name', 
        text_font_size='22px',
        text_color='black',
        # text_outline_color='white',
        text_font_style='bold',
        text_align='center',
        source=df
    )
    return p

def plot_notes(notes):
    frame = df[df['name'].isin(notes)].copy()
    return make_fig(frame)
    
# show(make_fig(df))

In [19]:
notes

['f#', 'g', 'g#', 'a', 'a#', 'b', 'c', 'c#', 'd', 'd#', 'e', 'f']

In [22]:
def get_chord(root_note, chord_type):
    """
    root_note (str): one of 'notes'
    chord_type (str): one of 'major', 'minor'
    """
    root_idx = notes.index(root_note)
    indexes = [root_idx]
    offset_dict = {
        'major': [4, 7],
        'minor': [3, 7],
        'third': [4],
        'fifth': [7],
    }
    offsets = offset_dict[chord_type]
    for offset in offsets:
        indexes.append((root_idx + offset) % n_notes)
    return [notes[i] for i in indexes]
        

In [33]:

def radial_plot(yo, title=None, show_text=True, size=400):
    """
    yo: DataFrame
    """
    
    n_wedges = len(yo)
    
    inner_radius = 0
    outer_radius = .5
    text_radius = outer_radius * 1.1 #(inner_radius + outer_radius) / 2
    angles = np.linspace(0, 2*pi, n_wedges + 1)
    
    middle_angles = (angles[:-1] + angles[1:] ) / 2
    
    yo['text_x'] = np.cos(middle_angles) * text_radius
    yo['text_y'] = np.sin(middle_angles) * text_radius
    
    yo['start_angle'] = angles[:-1]
    yo['end_angle'] = angles[1:]
    
    bound = 0.6
    p = figure(
        x_range=[-bound, bound], y_range=[-bound, bound],
        width=size,
        height=size,
        toolbar_location=None,
        x_axis_location=None,
        y_axis_location=None,
        title=title
        
    )
    p.annular_wedge(
        x=0, y=0,
        start_angle='start_angle',
        end_angle='end_angle',
        color='color',
        inner_radius=inner_radius,
        outer_radius=outer_radius,
        source=yo
    )
    if show_text:
        p.text(
            text='name',
            x='text_x',
            y='text_y',
            text_font_style='bold',
            text_align='center',
            source=yo
        )
    return p

In [34]:
def plot_chord(tonic, type):
    """
    tonic: str
        the root note
    type: str
        'major', 'minor', etc
    """
    chord_notes = get_chord(tonic, type)
    frame = df.loc[df['name'].isin(chord_notes)].copy()
    show(radial_plot(frame, title=f'{tonic} {type}'))


# Play

In [35]:
# source: https://muted.io/note-frequencies/
notes = [
    {'name': 'f#', 'note_frequency_Hz': 185},
    {'name': 'g',  'note_frequency_Hz': 196},
    {'name': 'g#', 'note_frequency_Hz': 207.65},
    {'name': 'a',  'note_frequency_Hz': 220},
    {'name': 'a#', 'note_frequency_Hz': 233.08},
    {'name': 'b',  'note_frequency_Hz': 246.94},
    {'name': 'c',  'note_frequency_Hz': 261.63},
    {'name': 'c#', 'note_frequency_Hz': 277.18},
    {'name': 'd',  'note_frequency_Hz': 293.66},
    {'name': 'd#', 'note_frequency_Hz': 311.13},
    {'name': 'e',  'note_frequency_Hz': 329.63},
    {'name': 'f',  'note_frequency_Hz': 349.23},
]

df = pd.DataFrame(notes)

df['light_frequency_THz'] = df['note_frequency_Hz'].apply(get_light_frequency)
df['color'] = df['light_frequency_THz'].apply(get_color)

df

,name,note_frequency_Hz,light_frequency_THz,color
0,f#,185.00,406.819302,"(158.67752253917945, 0.0, 0.0, 1.0)"
1,g,196.00,431.008558,"(207.6224790905554, 0.0, 0.0, 1.0)"
2,g#,207.65,456.627179,"(244.99771754242087, 0.0, 0.0, 1.0)"
3,a,220.00,483.785116,"(255.0, 159.14996191897876, 0.0, 1.0)"
4,a#,233.08,512.548340,"(255.0, 245.1884773698106, 0.0, 1.0)"
5,b,246.94,543.026803,"(197.7024800405969, 255.0, 0.0, 1.0)"
6,c,261.63,575.330454,"(101.44624977690603, 255.0, 0.0, 1.0)"
7,c#,277.18,609.525266,"(0.0, 255.0, 242.9480556450771, 1.0)"
8,d,293.66,645.765169,"(0.0, 177.56414477967783, 255.0, 1.0)"
9,d#,311.13,684.182105,"(43.98033984308385, 0.0, 252.27282488970076, 1.0)"


In [36]:
notes = list(df['name'])
n_notes = len(notes) # 12, chromatic scale
notes

['f#', 'g', 'g#', 'a', 'a#', 'b', 'c', 'c#', 'd', 'd#', 'e', 'f']

In [37]:
g_major = get_chord('g', 'major')
g_minor = get_chord('g', 'minor')
print(g_major)

['g', 'b', 'd']


In [69]:
show(radial_plot(df))

In [47]:
show(radial_plot(df.loc[df['name'].isin(['g', 'a', 'b', 'c', 'd', 'e', 'f'])].copy()))

In [50]:
show(radial_plot(df.loc[df['name'].isin(['g', 'c', 'd'])].copy()))

In [68]:
plot_chord('f', 'third')

In [58]:
show(radial_plot(df.loc[df['name'].isin(['a', 'd#'])].copy()))

In [16]:
show(radial_plot(df.copy(), show_text=True))

In [70]:
white_keys = ['g', 'a', 'b', 'c', 'd', 'e', 'f']
df_white_keys = df.loc[df['name'].isin(white_keys)].copy()
# show(radial_plot(df_white_keys))

# Circle of Fifths

In [ ]:
fifths = ['c', 'g', 'd', 'a', 'e', 'b', 'f#', 'c#', 'g#', 'd#', 'a#', 'f']
relative_minors = ['am', 